In [1]:
#Stufe 4 – Fortgeschrittene Aufgaben

# Vorbereitung übernahme aus Aufgaben davor
# Kundenanalyse Stuffe 2
# Einfache Verknüpfung
# 4 **Lade beide CSV-Dateien ein.**  

import pandas as pd
df_Kunden = pd.read_csv("../csv/Kunden.csv", dtype={"PLZ" : str})
df_Zuordnung = pd.read_csv("../csv/zuordnung_plz_ort.csv", dtype={"plz" :str})

# 5. **Führe beide Tabellen über `PLZ` zusammen.**  
# merge

merged = pd.merge(
    df_Kunden,
    df_Zuordnung.drop_duplicates(subset="plz"),
    how="left",
    left_on="PLZ",
    right_on="plz"
)

print(len(df_Zuordnung))
print(len(df_Kunden))
print(len(merged))

12854
2500
2500


In [3]:
print(merged.columns)
print(list(merged.columns))

Index(['Vorname', 'Nachname', 'PLZ', 'Ort', 'osm_id', 'ags', 'ort', 'plz',
       'landkreis', 'bundesland'],
      dtype='object')
['Vorname', 'Nachname', 'PLZ', 'Ort', 'osm_id', 'ags', 'ort', 'plz', 'landkreis', 'bundesland']


In [5]:
# 12. **Erzeuge einen Report mit:**  
#    - Bundesland  
#   - Anzahl Kunden  
#   - Anzahl Orte  
#   - Häufigste PLZ je Bundesland

# 12. Report je Bundesland
report = merged.groupby("bundesland").agg(
    anzahl_kunden=("Vorname", "count"),
    anzahl_orte=("Ort", "nunique"),
    haeufigste_plz=("PLZ", lambda x: x.mode().iloc[0] if not x.mode().empty else None)
)
print(report)


                        anzahl_kunden  anzahl_orte haeufigste_plz
bundesland                                                       
Baden-Württemberg                 261          217          71254
Bayern                            442          375          82024
Berlin                             37            1          10249
Brandenburg                        64           60          15306
Bremen                              9            2          28359
Hamburg                            15            1          22115
Hessen                             98           75          61440
Mecklenburg-Vorpommern            163          142          17209
Niedersachsen                     188          163          21423
Nordrhein-Westfalen               173          101          32469
Rheinland-Pfalz                   445          397          54518
Saarland                           11            8          66640
Sachsen-Anhalt                     34           29          39291
Schleswig-

In [6]:
# Anzahl aller eindeutigen Orte im gesamten DataFrame
anzahl_orte_gesamt = merged["Ort"].nunique()
print("Anzahl eindeutiger Orte insgesamt:", anzahl_orte_gesamt)

# Anzahl Datensätze ohne Bundesland
anzahl_ohne_bundesland = merged[merged["bundesland"].isna()]["Ort"].nunique()
print("Anzahl eindeutiger Orte ohne Bundesland:", anzahl_ohne_bundesland)

Anzahl eindeutiger Orte insgesamt: 2012
Anzahl eindeutiger Orte ohne Bundesland: 174


In [7]:
# der Report greift nicht auf alle Daten zu weil es Kunden ohne Bundesland gibt


In [ ]:
# 13. **Exportiere Kunden gruppiert nach Bundesland als JSON-Datei.**  
#    Format:  
#   ```json
#    {
#     "Berlin": [ {…}, {…} ],
#     "NRW": [ {…} ]
#   }
import json

# 13. JSON-Export Grupierung nach Bundesland und Ditionary
kunden_json = df.groupby("bundesland").apply(lambda d: d.to_dict(orient="records")).to_dict()

# Als Datei speichern (optional)

with open("kunden_nach_bundesland.json", "w") as f:
    json.dump(kunden_json, f, indent=2)
